In [8]:
import pandas as pd
table = pd.read_csv("../../Module1/0-Data/recipeData.csv",  encoding="latin-1")
pd.set_option("display.max_columns", len(table.columns))
table.head()

,BeerID,Name,URL,Style,StyleID,Size(L),OG,FG,ABV,IBU,Color,BoilSize,BoilTime,BoilGravity,Efficiency,MashThickness,SugarScale,BrewMethod,PitchRate,PrimaryTemp,PrimingMethod,PrimingAmount,UserId
0,1,Vanilla Cream Ale,/homebrew/recipe/view/1633/vanilla-cream-ale,Cream Ale,45,21.77,1.055,1.013,5.48,17.65,4.83,28.39,75,1.038,70.0,NaN,Specific Gravity,All Grain,NaN,17.78,corn sugar,4.5 oz,116.0
1,2,Southern Tier Pumking clone,/homebrew/recipe/view/16367/southern-tier-pumk...,Holiday/Winter Special Spiced Beer,85,20.82,1.083,1.021,8.16,60.65,15.64,24.61,60,1.070,70.0,NaN,Specific Gravity,All Grain,NaN,NaN,NaN,NaN,955.0
2,3,Zombie Dust Clone - EXTRACT,/homebrew/recipe/view/5920/zombie-dust-clone-e...,American IPA,7,18.93,1.063,1.018,5.91,59.25,8.98,22.71,60,NaN,70.0,NaN,Specific Gravity,extract,NaN,NaN,NaN,NaN,NaN
3,4,Zombie Dust Clone - ALL GRAIN,/homebrew/recipe/view/5916/zombie-dust-clone-a...,American IPA,7,22.71,1.061,1.017,5.80,54.48,8.50,26.50,60,NaN,70.0,NaN,Specific Gravity,All Grain,NaN,NaN,NaN,NaN,NaN
4,5,Bakke Brygg Belgisk Blonde 50 l,/homebrew/recipe/view/89534/bakke-brygg-belgis...,Belgian Blond Ale,20,50.00,1.060,1.010,6.48,17.84,4.57,60.00,90,1.050,72.0,NaN,Specific Gravity,All Grain,NaN,19.00,Sukkerlake,6-7 g sukker/l,18325.0


In [9]:
drop = ["BeerID", "Name", "URL", "Style", "UserId", "PrimingMethod", "PrimingAmount"]
table.drop(drop, axis=1, inplace=True)

styles = table["StyleID"].value_counts()
above = styles.loc[styles > 1000]
table = table.loc[table["StyleID"].isin(above.index)]

sugar_encode = pd.get_dummies(table["SugarScale"], dtype=float)
brew_encode = pd.get_dummies(table["BrewMethod"], dtype=float)
concatenated = pd.concat([table, sugar_encode, brew_encode], axis=1)
concatenated.drop("SugarScale", axis=1, inplace=True)
concatenated.drop("BrewMethod", axis=1, inplace=True)

pd.set_option("display.max_columns", len(concatenated.columns))
table = concatenated
table.head()

,StyleID,Size(L),OG,FG,ABV,IBU,Color,BoilSize,BoilTime,BoilGravity,Efficiency,MashThickness,PitchRate,PrimaryTemp,Plato,Specific Gravity,All Grain,BIAB,Partial Mash,extract
2,7,18.93,1.063,1.018,5.91,59.25,8.98,22.71,60,NaN,70.0,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,1.0
3,7,22.71,1.061,1.017,5.80,54.48,8.50,26.50,60,NaN,70.0,NaN,NaN,NaN,0.0,1.0,1.0,0.0,0.0,0.0
5,10,24.61,1.055,1.013,5.58,40.12,8.00,29.34,70,1.047,79.0,NaN,1.0,NaN,0.0,1.0,1.0,0.0,0.0,0.0
6,86,22.71,1.072,1.018,7.09,268.71,6.33,30.28,90,NaN,75.0,NaN,NaN,NaN,0.0,1.0,1.0,0.0,0.0,0.0
9,86,20.82,1.080,1.017,8.22,93.02,8.29,28.39,60,1.058,70.0,NaN,NaN,21.11,0.0,1.0,1.0,0.0,0.0,0.0


In [10]:
fill = {"BoilGravity": table["BoilGravity"].median(), "MashThickness": table["MashThickness"].median(
), "PrimaryTemp": table["PrimaryTemp"].median()}
table = table.fillna(value=fill)

In [11]:
table.isnull().sum()/len(table)*100

StyleID              0.00000
Size(L)              0.00000
OG                   0.00000
FG                   0.00000
ABV                  0.00000
IBU                  0.00000
Color                0.00000
BoilSize             0.00000
BoilTime             0.00000
BoilGravity          0.00000
Efficiency           0.00000
MashThickness        0.00000
PitchRate           55.18575
PrimaryTemp          0.00000
Plato                0.00000
Specific Gravity     0.00000
All Grain            0.00000
BIAB                 0.00000
Partial Mash         0.00000
extract              0.00000
dtype: float64

In [12]:
x_train = table[table["PitchRate"].notnull()]
x_train.drop("PitchRate", axis=1, inplace=True)
y_train = table[table["PitchRate"].notnull()]["PitchRate"]
x_fill = table[table["PitchRate"].isnull()]
x_fill.drop("PitchRate", axis=1, inplace=True)
y_fill = table[table["PitchRate"].isnull()]["PitchRate"]


C:\Users\Matheus\AppData\Local\Temp\ipykernel_4492\2433983694.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train.drop("PitchRate", axis=1, inplace=True)
C:\Users\Matheus\AppData\Local\Temp\ipykernel_4492\2433983694.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_fill.drop("PitchRate", axis=1, inplace=True)


In [13]:
from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor()
model.fit(x_train, y_train)

y_fill = model.predict(x_fill)

In [14]:
table.PitchRate[table.PitchRate.isnull()] = y_fill
table

C:\Users\Matheus\AppData\Local\Temp\ipykernel_4492\2380314759.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table.PitchRate[table.PitchRate.isnull()] = y_fill


,StyleID,Size(L),OG,FG,ABV,IBU,Color,BoilSize,BoilTime,BoilGravity,Efficiency,MashThickness,PitchRate,PrimaryTemp,Plato,Specific Gravity,All Grain,BIAB,Partial Mash,extract
2,7,18.93,1.063,1.018,5.91,59.25,8.98,22.71,60,1.047,70.0,1.5,0.35,20.00,0.0,1.0,0.0,0.0,0.0,1.0
3,7,22.71,1.061,1.017,5.80,54.48,8.50,26.50,60,1.047,70.0,1.5,0.35,20.00,0.0,1.0,1.0,0.0,0.0,0.0
5,10,24.61,1.055,1.013,5.58,40.12,8.00,29.34,70,1.047,79.0,1.5,1.00,20.00,0.0,1.0,1.0,0.0,0.0,0.0
6,86,22.71,1.072,1.018,7.09,268.71,6.33,30.28,90,1.047,75.0,1.5,0.75,20.00,0.0,1.0,1.0,0.0,0.0,0.0
9,86,20.82,1.080,1.017,8.22,93.02,8.29,28.39,60,1.058,70.0,1.5,0.35,21.11,0.0,1.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73853,7,18.93,1.073,1.018,7.15,0.00,6.01,22.71,60,1.060,70.0,1.5,0.50,20.00,0.0,1.0,0.0,1.0,0.0,0.0
73854,10,18.93,1.064,1.016,6.26,0.00,9.60,28.39,60,1.042,75.0,1.5,0.35,20.00,0.0,1.0,0.0,1.0,0.0,0.0
73855,7,22.71,1.065,1.014,6.71,49.35,4.73,15.14,30,1.098,55.0,1.5,0.50,20.00,0.0,1.0,0.0,0.0,1.0,0.0
73859,92,24.00,1.051,1.014,4.82,25.65,14.12,28.00,60,1.043,72.0,1.5,0.50,18.00,0.0,1.0,1.0,0.0,0.0,0.0
